In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import imageio

# Neural Network

In [3]:


# Define the neural network architecture
class FCN(nn.Module):
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[nn.Linear(N_INPUT, N_HIDDEN), activation()])
        self.fch = nn.Sequential(*[nn.Sequential(*[nn.Linear(N_HIDDEN, N_HIDDEN), activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
        
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

# Define the true solution for the 2D diffusion equation
def u_true(x, y, t):
    return np.exp(-((x - 0.5 - 0.1 * t)**2 + (y - 0.5 - 0.1 * t)**2) / (2 * 0.1**2))

# Generate observational input data
nx, ny, nt = 10, 10, 2
xr = np.linspace(0, 1, nx)
yr = np.linspace(0, 1, ny)
tr = np.linspace(0, nt-1, nt).T
xrmesh, yrmesh, trmesh = np.meshgrid(xr, yr, tr)
ur = u_true(xrmesh, yrmesh, trmesh)

rin_data = np.stack((xrmesh, yrmesh, trmesh), axis=-1).reshape(-1, 3)
rin_data = torch.tensor(rin_data).float()
rout_data = torch.tensor(ur).float().reshape(-1, 1)

# Fit a neural network to the observational data
torch.manual_seed(123)
model = FCN(3, 1, 32, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for i in range(5000):
    optimizer.zero_grad()
    uh = model(rin_data)
    loss = torch.mean((uh - rout_data)**2)
    loss.backward()
    optimizer.step()
    if (i+1) % 1000 == 0:
        print(f'Epoch: {i+1}/5000, Loss: {loss.item()}')

# Plot and compare the results
nt = 10
xr = np.linspace(0, 1.2, nx)
yr = np.linspace(0, 1.2, ny)
tr = np.linspace(0, nt-1, nt).T
xrmesh, yrmesh, trmesh = np.meshgrid(xr, yr, tr)

rin_test = np.stack((xrmesh, yrmesh, trmesh), axis=-1).reshape(-1, 3)
rin_test = torch.tensor(rin_test).float()

cwd = os.getcwd()
true_plot_dir = os.path.join(cwd, 'diffusion', 'true_plot')
nn_plot_dir = os.path.join(cwd, 'diffusion', 'nn_plot')
os.makedirs(nn_plot_dir, exist_ok=True)
os.makedirs(true_plot_dir, exist_ok=True)

for tt in range(nt):
    ur = model(rin_test.float()).detach().numpy().reshape(nx, ny, nt)
    ut = u_true(xrmesh, yrmesh, trmesh)

    plt.contourf(xrmesh[:, :, 0], yrmesh[:, :, 0], ur[:, :, tt])
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('NN Data at t=' + str(tt))
    plt.colorbar()
    plt.savefig(nn_plot_dir + f'/nn_plot_{tt}.png')
    plt.close()

    plt.contourf(xrmesh[:, :, 0], yrmesh[:, :, 0], ut[:, :, tt])
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Actual Data at t=' + str(tt))
    plt.colorbar()
    plt.savefig(true_plot_dir + f'/true_plot_{tt}.png')
    plt.close()

# Save the plots as GIFs
for _ in ['nn_plot', 'true_plot']:
    plot_dir = os.path.join(cwd, 'diffusion', _)
    plot_files = sorted([os.path.join(plot_dir, file) for file in os.listdir(plot_dir)])
    images = [imageio.imread(file) for file in plot_files]
    output_file = os.path.join(cwd, f"diffusion_{_}.gif")
    imageio.mimsave(output_file, images, duration=1, loop=0)

Epoch: 1000/5000, Loss: 5.9737842093454674e-05
Epoch: 2000/5000, Loss: 2.1684474631911144e-05
Epoch: 3000/5000, Loss: 9.965334356820676e-06
Epoch: 4000/5000, Loss: 5.836269792780513e-06
Epoch: 5000/5000, Loss: 4.321484539104858e-06


/tmp/ipykernel_37868/1461266421.py:85: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(file) for file in plot_files]


# Physics-Informed Neural Network

In [8]:
# randomly sample points in (x,y)-t space, where x in [0,1] and t in [0,5]
np.random.seed(123)
rin_physics = np.random.uniform(size=(nx*ny*nt,3))
rin_physics[:,2] *= nt
rin_physics = torch.tensor(rin_physics).float().requires_grad_(True)

# Fit a neural network to the observational data
torch.manual_seed(123)
pinn = FCN(3, 1, 32, 3)
optimizer = torch.optim.Adam(pinn.parameters(), lr=1e-3)
for i in range(5000):
    optimizer.zero_grad()
    uh = pinn(rin_data)
    loss1 = torch.mean((uh - rout_data)**2)
    uhp = pinn(rin_physics)
    grad = torch.autograd.grad(uhp, rin_physics, grad_outputs=torch.ones_like(uhp), create_graph=True)[0]
    dudx = grad[:, 0]
    dudy = grad[:, 1]
    dudt = grad[:, 2]
    dudxx  = torch.autograd.grad(dudx, rin_physics, grad_outputs=torch.ones_like(dudx), create_graph=True)[0]
    dudyy  = torch.autograd.grad(dudy, rin_physics, grad_outputs=torch.ones_like(dudy), create_graph=True)[0]
    physics = dudt - 0.01*(dudxx[:,0] + dudyy[:,0])
    loss2 = torch.mean(physics**2)
    loss = 2*loss1 + loss2
    loss.backward()
    optimizer.step()
    if (i+1) % 1000 == 0:
        print(f'Epoch: {i+1}/5000, Loss: {loss.item()}')

# Plot and compare the results
nt = 10
xr = np.linspace(0, 1.2, nx)
yr = np.linspace(0, 1.2, ny)
tr = np.linspace(0, nt-1, nt).T
xrmesh, yrmesh, trmesh = np.meshgrid(xr, yr, tr)

rin_test = np.stack((xrmesh, yrmesh, trmesh), axis=-1).reshape(-1, 3)
rin_test = torch.tensor(rin_test).float()

cwd = os.getcwd()
true_plot_dir = os.path.join(cwd, 'diffusion', 'true_plot')
nn_plot_dir = os.path.join(cwd, 'diffusion', 'pinn_plot')
os.makedirs(nn_plot_dir, exist_ok=True)
os.makedirs(true_plot_dir, exist_ok=True)

for tt in range(nt):
    ur = pinn(rin_test.float()).detach().numpy().reshape(nx, ny, nt)
    ut = u_true(xrmesh, yrmesh, trmesh)

    plt.contourf(xrmesh[:, :, 0], yrmesh[:, :, 0], ur[:, :, tt])
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('PINN Data at t=' + str(tt))
    plt.colorbar()
    plt.savefig(nn_plot_dir + f'/pinn_plot_{tt}.png')
    plt.close()

    plt.contourf(xrmesh[:, :, 0], yrmesh[:, :, 0], ut[:, :, tt])
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Actual Data at t=' + str(tt))
    plt.colorbar()
    plt.savefig(true_plot_dir + f'/true_plot_{tt}.png')
    plt.close()

# Save the plots as GIFs
for _ in ['pinn_plot', 'true_plot']:
    plot_dir = os.path.join(cwd, 'diffusion', _)
    plot_files = sorted([os.path.join(plot_dir, file) for file in os.listdir(plot_dir)])
    images = [imageio.imread(file) for file in plot_files]
    output_file = os.path.join(cwd, f"diffusion_{_}.gif")
    imageio.mimsave(output_file, images, duration=1, loop=0)


Epoch: 1000/5000, Loss: 0.028076522052288055
Epoch: 2000/5000, Loss: 0.026430463418364525
Epoch: 3000/5000, Loss: 0.0017716442234814167
Epoch: 4000/5000, Loss: 0.0002734983281698078
Epoch: 5000/5000, Loss: 0.00013975291221868247


/tmp/ipykernel_37868/3696269741.py:70: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(file) for file in plot_files]
